In [101]:
import pandas as pd
import re
import requests

from bs4 import BeautifulSoup

URL_ROOT = 'https://en.wiktionary.org'

def get_soup(path):
    res = requests.get(f"{URL_ROOT}{path}")
    return BeautifulSoup(res.text)

In [105]:
def get_category_page_ranges(start_url):
    print(start_url)
    next_url = start_url
    soup = get_soup(start_url)
    next_link = soup.find('a', string='next page')
    while next_link:
        print(next_link['href'])
        soup = get_soup(next_link['href'])
        next_link = soup.find('a', string='next page')
    print("Done!")


#get_category_page_ranges("/wiki/Category:Vietnamese_lemmas")
get_category_page_ranges("/wiki/Category:Vietnamese_adverbs")

/wiki/Category:Vietnamese_adverbs
/w/index.php?title=Category:Vietnamese_adverbs&pagefrom=D%EF%80%80ANG%21+HOANG%21%0A%C4%91%C3%A0ng+ho%C3%A0ng#mw-pages
/w/index.php?title=Category:Vietnamese_adverbs&pagefrom=LO%26+MO%26%0Al%E1%BB%8D+m%E1%BB%8D#mw-pages
/w/index.php?title=Category:Vietnamese_adverbs&pagefrom=SU%EF%80%80C%26%0As%E1%BB%B1c#mw-pages
/w/index.php?title=Category:Vietnamese_adverbs&pagefrom=%E4%B8%8003%E8%BE%B509%0A%E4%B8%8D%E9%81%8E#mw-pages
Done!


In [ ]:
soup = get_soup("/wiki/Category:Vietnamese_lemmas")

In [93]:
[a['href'] for a in soup.css.select('div#mw-pages li a')]

['/wiki/9x',
 '/wiki/A',
 '/wiki/a',
 '/wiki/a_c%C3%B2ng',
 '/wiki/A_Di_%C4%90%C3%A0',
 '/wiki/A_Di_%C4%90%C3%A0_Ph%E1%BA%ADt',
 '/wiki/a_dua',
 '/wiki/a_giao',
 '/wiki/a_ha',
 '/wiki/a_ho%C3%A0n',
 '/wiki/A_La_H%C3%A1n',
 '/wiki/A_L%E1%BB%8Bch_S%C6%A1n',
 '/wiki/A_L%E1%BB%8Bch_S%C6%A1n_%C4%90%E1%BA%AFc_L%E1%BB%99',
 '/wiki/a_l%C3%B4',
 '/wiki/a_m%C3%B3c',
 '/wiki/a_phi%E1%BA%BFn',
 '/wiki/A_Ph%C3%BA_H%C3%A3n',
 '/wiki/A_Q',
 '/wiki/a_t%C3%B2ng',
 '/wiki/A_Tu_La',
 '/wiki/%C3%A0',
 '/wiki/%C3%A0_th%E1%BA%BF_%C3%A0',
 '/wiki/%C3%A0_u%C3%B4m',
 '/wiki/%E1%BA%A2',
 '/wiki/%E1%BA%A3',
 '/wiki/%E1%BA%A2_R%E1%BA%ADp',
 '/wiki/%C3%A1',
 '/wiki/%C3%81',
 '/wiki/%C3%81_C%C4%83n_%C4%90%C3%ACnh',
 '/wiki/%C3%81_Ch%C3%A2u',
 '/wiki/%C3%81_ch%C3%A2u',
 '/wiki/%C3%81_%C4%90%C3%B4ng',
 '/wiki/%C3%A1_%C4%91%C3%B9',
 '/wiki/%C3%A1_h%E1%BA%ADu',
 '/wiki/%C3%A1_ho%C3%A1_th%E1%BA%A1ch',
 '/wiki/%C3%A1_h%C3%B3a_th%E1%BA%A1ch',
 '/wiki/%C3%A1_kh%E1%BA%A9u',
 '/wiki/%C3%A1_kh%C3%B4i',
 '/wiki/%C3%A1_kim',
 '